In [1]:
import pyvisa
import plotly.graph_objects as go
import numpy as np
import time

In [2]:
rm = pyvisa.ResourceManager()
rm.list_resources()
Keithley = rm.open_resource("GPIB0::24::INSTR")
Keithley.timeout = 1000000
print(Keithley.query("*IDN?"))
Datalist = []

c:\Users\ziwzh166\AppData\Local\miniconda3\Lib\site-packages\pyvisa_py\tcpip.py:121: UserWarning: TCPIP::hislip resource discovery requires the zeroconf package to be installed... try 'pip install zeroconf'
  warnings.warn(


KEITHLEY INSTRUMENTS INC.,MODEL 6430,4487834,C27   Jul 12 2004 15:47:33/B01  /H/C



In [101]:
start_voltage = -0.1
end_voltage = 0.7
step_voltage = 0.005

# Determine the number of decimal places in step_voltage
decimal_places = len(str(step_voltage).split(".")[1])

# Determine the direction of the sweep
if start_voltage < end_voltage:
    # Generate the increasing sequence
    voltage_list = [
        start_voltage + i * step_voltage
        for i in range(int((end_voltage - start_voltage) / step_voltage) + 1)
    ]
else:
    # Generate the decreasing sequence
    voltage_list = [
        start_voltage - i * step_voltage
        for i in range(int((start_voltage - end_voltage) / step_voltage) + 1)
    ]

# Return to the start voltage
voltage_list += voltage_list[-2::-1]  # Avoid duplicating the end_voltage

# Format each voltage to have the same number of decimal places
voltage_list = [f"{v:.{decimal_places}f}" for v in voltage_list]

# Split the voltage list into chunks of 100 data points
chunk_size = 100
voltage_chunks = [
    voltage_list[i : i + chunk_size] for i in range(0, len(voltage_list), chunk_size)
]
print(len(voltage_chunks))

print(f"The number of data points is {len(voltage_list)}")

4
The number of data points is 319


In [ ]:
Keithley.write(":*RST")  # Reset the instrument
Keithley.write(":SOUR:FUNC VOLT")  # Set source function to voltage
Keithley.write(":SOUR:VOLT:MODE LIST")  # Set source mode to list
Keithley.write(":DISP:ENAB OFF")
chunk1 = voltage_chunks[0]
chunk1_str = ",".join(chunk1)
Keithley.write(f":SOUR:LIST:VOLT {chunk1_str}")
# Append each chunk to the devices
for chunk in voltage_chunks[1::]:
    voltage_list_str = ",".join(chunk)
    Keithley.write(f":SOUR:LIST:VOLT:APP {voltage_list_str}")

Keithley.write(":SENS:CURR:PROT 1e-6")  # Set current compliance
Keithley.write(":DISP:ENAB ON")  # Disable the display
Keithley.write(":SENS:CURR:NPLC 0.1")  # Set integration time
Keithley.write(":SENS:CURR:RANG 1e-10")  # Set current range
Keithley.write(
    f":TRIG:COUN {len(voltage_list)}"
)  # Set trigger count to the number of voltages in the list

Keithley.write(":FORM:ELEM CURR")  # Specify to measure only current
Keithley.write(":OUTP ON")  # Enable the output
Keithley.write(":TRAC:CLE")  # Clear the buffer
Keithley.write(":INIT")  # Initiate the measurement
Keithley.write("*WAI")  # Wait for the measurement to complete

6

In [108]:
data = Keithley.query(":READ?")  # Retrieve the measured data

In [23]:
data

'-5.957832E-13,-5.549499E-13,-5.292939E-13,-5.410079E-13,-5.298350E-13,-5.174849E-13,-5.105865E-13,-5.129919E-13,-5.048467E-13,-5.168570E-13,-5.139967E-13,-5.157646E-13,-5.009765E-13,-5.082025E-13,-5.086241E-13,-4.861112E-13,-5.058895E-13,-4.949302E-13,-4.756882E-13,-4.809451E-13,-4.893791E-13,-4.806442E-13,-4.791381E-13,-4.809411E-13,-4.844712E-13,-4.748907E-13,-4.882047E-13,-4.790918E-13,-4.744312E-13,-4.582514E-13,-4.608615E-13,-4.520485E-13,-4.502816E-13,-4.480543E-13,-4.460766E-13,-4.354094E-13,-4.070564E-13,-4.393978E-13,-4.267916E-13,-4.378058E-13,-4.291469E-13,-4.376373E-13,-4.232748E-13,-4.319238E-13,-4.315419E-13,-4.228017E-13,-4.291024E-13,-4.131397E-13,-4.239828E-13,-4.261661E-13,-4.157482E-13,-4.171788E-13,-4.021850E-13,-4.107484E-13,-4.067611E-13,-4.112888E-13,-3.912196E-13,-3.965127E-13,-3.825663E-13,-3.964702E-13,-3.913442E-13,-3.869367E-13,-3.916749E-13,-3.926092E-13,-3.834071E-13,-3.864698E-13,-3.860886E-13,-3.794555E-13,-3.858448E-13,-3.785321E-13,-3.715589E-13,-3.79

In [109]:
# Split the data string by commas
data_list = data.split(",")

# Process and print the data
data_values = []
for value in data_list:
    value = value.replace("+", "").strip()
    if value:
        try:
            data_values.append(float(value))
        except ValueError:
            print(f"Could not convert value to float: {value}")


# Iterate with a step of 2
for voltage, current in zip(voltage_list, data_values):
    print(f"Voltage: {voltage} V, Current: {current} A")

Voltage: -0.100 V, Current: 1.961661e-13 A
Voltage: -0.095 V, Current: 5.827493e-13 A
Voltage: -0.090 V, Current: 6.578831e-13 A
Voltage: -0.085 V, Current: 7.702554e-13 A
Voltage: -0.080 V, Current: 8.252704e-13 A
Voltage: -0.075 V, Current: 8.980244e-13 A
Voltage: -0.070 V, Current: 9.668105e-13 A
Voltage: -0.065 V, Current: 1.048262e-12 A
Voltage: -0.060 V, Current: 1.084421e-12 A
Voltage: -0.055 V, Current: 1.145801e-12 A
Voltage: -0.050 V, Current: 1.210689e-12 A
Voltage: -0.045 V, Current: 1.268724e-12 A
Voltage: -0.040 V, Current: 1.319135e-12 A
Voltage: -0.035 V, Current: 1.383253e-12 A
Voltage: -0.030 V, Current: 1.435474e-12 A
Voltage: -0.025 V, Current: 1.488906e-12 A
Voltage: -0.020 V, Current: 1.543843e-12 A
Voltage: -0.015 V, Current: 1.593496e-12 A
Voltage: -0.010 V, Current: 1.645136e-12 A
Voltage: -0.005 V, Current: 1.704097e-12 A
Voltage: 0.000 V, Current: 1.754025e-12 A
Voltage: 0.005 V, Current: 1.799996e-12 A
Voltage: 0.010 V, Current: 1.87294e-12 A
Voltage: 0.015 

In [110]:
filename = "PtNE1_0325_Clean_2mMFcMeOH_KCl_AFTSG03_Poli"

In [111]:
import plotly.graph_objects as go

fig = go.Figure(
    data=go.Scatter(
        x=[float(v) for v in voltage_list],
        y=data_values,
    )
)
fig.update_layout(
    title=filename,
    xaxis_title="Voltage (V)",
    yaxis_title="Current (A)",
    template="simple_white",
    xaxis=dict(tickformat=".1f", dtick=0.1),
)

fig.show()
fig.write_html("./CV_" + filename + ".html")

In [90]:
import numpy as np

header = ["Potential (V)", "Current (A)"]
# Ensure start_voltage is less than end_voltage
if start_voltage > end_voltage:
    start_voltage, end_voltage = end_voltage, start_voltage

voltage_values = np.arange(start_voltage, end_voltage + step_voltage, step_voltage)
min_length = min(len(voltage_values), len(data_values))
voltage_values = voltage_values[:min_length]
current_values = data_values[:min_length]
Datatxt = np.column_stack((voltage_values, current_values))
# change saved file name here
txtname = "CV_" + filename + ".txt"
np.savetxt(txtname, Datatxt, delimiter="\t", header="\t".join(header), comments="")

In [40]:
# Reset the instrument and configure it
Keithley.write(":*RST")  # Reset the instrument
Keithley.write(":SOUR:FUNC VOLT")  # Set source function to voltage
Keithley.write(":SOUR:VOLT:MODE LIST")  # Set source mode to list
chunk1 = voltage_chunks[0]
chunk1_str = ",".join(chunk1)
Keithley.write(f":SOUR:LIST:VOLT {chunk1_str}")
# Append each chunk to the devices
for chunk in voltage_chunks[1::]:
    voltage_list_str = ",".join(chunk)
    Keithley.write(f":SOUR:LIST:VOLT:APP {voltage_list_str}")

Keithley.write(":SENS:CURR:PROT 1e-6")  # Set current compliance to 10 μA
Keithley.write(":DISP:ENAB OFF")  # Disable the display
Keithley.write(":SENS:CURR:NPLC 0.1")  # Set integration time to 1 PLC
Keithley.write(":SENS:CURR:RANG 1e-9")  # Set current range to 1 pA
Keithley.write(
    f":TRIG:COUN {len(voltage_list)}"
)  # Set trigger count to the number of voltages in the list

Keithley.write(":FORM:ELEM CURR, TIME")  # Specify to measure current and time
Keithley.write(":TRAC:CLE")  # Clear the buffer
Keithley.write(":TRAC:POIN 2000")  # Set buffer size to 2000 points
Keithley.write(":TRAC:FEED SENS")  # Feed measurement data to the buffer
Keithley.write(":TRAC:FEED:CONT NEXT")  # Continuously feed data to the buffer

Keithley.write(":OUTP ON")  # Enable the output
Keithley.write(":INIT")  # Initiate the measurement
Keithley.write("*WAI")  # Wait for the measurement to complete

# Retrieve the data from the buffer
data = Keithley.query(":TRAC:DATA?")

# Split the data into current and time
data_points = data.split(",")
currents = data_points[0::2]
timestamps = data_points[1::2]

# Print the data with the timestamps
for current, timestamp in zip(currents, timestamps):
    print(f"Current: {current}, Time: {timestamp}")

Current: +3.979386E-13, Time: +0.000000E+00
Current: +8.296642E-14, Time: +6.640625E-02
Current: -2.804433E-13, Time: +1.328125E-01
Current: -4.333385E-13, Time: +1.982422E-01
Current: -4.735499E-13, Time: +2.646484E-01
Current: -4.737182E-13, Time: +3.300781E-01
Current: -4.708759E-13, Time: +3.964844E-01
Current: -4.962033E-13, Time: +4.619141E-01
Current: -5.424223E-13, Time: +5.283203E-01
Current: -5.908173E-13, Time: +5.937500E-01
Current: -6.357855E-13, Time: +6.601563E-01
Current: -6.858177E-13, Time: +7.255859E-01
Current: -7.419686E-13, Time: +7.919922E-01
Current: -7.648480E-13, Time: +8.574219E-01
Current: -7.876914E-13, Time: +9.238281E-01
Current: -8.256239E-13, Time: +9.892578E-01
Current: -8.841960E-13, Time: +1.055664E+00
Current: -9.398894E-13, Time: +1.121094E+00
Current: -1.000995E-12, Time: +1.187500E+00
Current: -1.051520E-12, Time: +1.252930E+00
Current: -1.088435E-12, Time: +1.319336E+00
Current: -1.155466E-12, Time: +1.384766E+00
Current: -1.236099E-12, Time: +1

In [41]:
timeStamps_float = np.array([float(i) for i in timestamps])
Diff = np.diff(timeStamps_float)
values, counts = np.unique(Diff, return_counts=True)
print(f"the average time step is {values[np.argmax(counts)]*1e3} ms")
print(
    f"the scan rate is {step_voltage/values[np.argmax(counts)]} V/s, {step_voltage/values[np.argmax(counts)]*1e3} mV/s"
)

the average time step is 33.200000000000784 ms
the scan rate is 0.07530120481927532 V/s, 75.30120481927533 mV/s
